<a href="https://colab.research.google.com/github/LEEYOUNHO-HUFS/Infinite-Challenge/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

ModuleNotFoundError: ignored

In [ ]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 460kB 43.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK

rawdata_downloaded = drive.CreateFile({'id': '1CrNHY-9x43QcoE6AlKK7yCOZ9fXiBstR'})
rawdata_downloaded.GetContentFile('rawdata.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,문장번호,발화자,대상자,본문,자막,상황분류,설명
0,1,유재석,모두,"우리 각자 오늘 저 법원을 배경으로 해서 쭉 섰는데, 우리가 사실 이 법에 대해서 ...",오늘의 녹화 장소는 모의법정,진행,유재석이 진행자로서 프로그램을 진행하고 있다.
1,2,유재석,모두,"그래서 오늘 저희가말이죠, 여러가지들을 조금 알아 보고 과연 우리들은 평소에 잘못한...",NaN,진행,NaN
2,3,정형돈,유재석,"저, 길이 하나 있어요!",NaN,폭로,NaN
3,4,유재석,정형돈,뭐요?,NaN,폭로,NaN
4,5,정형돈,유재석,노상방뇨죄.,노상 방뇨 죄,폭로,NaN
...,...,...,...,...,...,...,...
1738,1739,노홍철,판사,감사합니다.,이제 화해 권고안 받아들일 차례,재판,NaN
1739,1740,유재석,장진영,저 그럼… 장변호사님.,NaN,재판,NaN
1740,1741,장진영,모두,우리끼리 벌칙 정해야 할 것 같은데요 이제?,NaN,재판,NaN
1741,1742,정형돈,장진영,누가 이긴거에요 그럼?,NaN,재판,NaN


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[3] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('무승부', 'Noun'), ('죠', 'Josa'), ('?', 'Punctuation')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,무승부,Noun,장진영,1
1,죠,Josa,장진영,1
2,?,Punctuation,장진영,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[        0            1 type  count
 0      우리         Noun  유재석      1
 1      각자         Noun  유재석      1
 2      오늘         Noun  유재석      1
 3       저         Noun  유재석      1
 4      법원         Noun  유재석      1
 5       을         Josa  유재석      1
 6      배경         Noun  유재석      1
 7      으로         Josa  유재석      1
 8      해서         Verb  유재석      1
 9       쭉         Noun  유재석      1
 10    섰는데         Verb  유재석      1
 11      ,  Punctuation  유재석      1
 12     우리         Noun  유재석      1
 13      가         Josa  유재석      1
 14     사실         Noun  유재석      1
 15      이         Noun  유재석      1
 16      법         Noun  유재석      1
 17      에         Josa  유재석      1
 18     대해         Noun  유재석      1
 19      서         Josa  유재석      1
 20     평소         Noun  유재석      1
 21      에         Josa  유재석      1
 22  알고있는게         Verb  유재석      1
 23    의외로       Adverb  유재석      1
 24     많이       Adverb  유재석      1
 25    없어요    Adjective  유재석      1
 26      .  Punctuation  유재석

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,우리,Noun,유재석,1
1,각자,Noun,유재석,1
2,오늘,Noun,유재석,1
3,저,Noun,유재석,1
4,법원,Noun,유재석,1
...,...,...,...,...
2,그럼,Adjective,정형돈,1
3,?,Punctuation,정형돈,1
0,무승부,Noun,장진영,1
1,죠,Josa,장진영,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,우리,Noun,유재석,1
1,각자,Noun,유재석,1
2,오늘,Noun,유재석,1
3,저,Noun,유재석,1
4,법원,Noun,유재석,1
...,...,...,...,...
2,그럼,Adjective,정형돈,1
3,?,Punctuation,정형돈,1
0,무승부,Noun,장진영,1
1,죠,Josa,장진영,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소     품사           분류    
!       Punctuation  길         72
                     길씨 어머님     3
                     김제동       12
                     김태호        2
                     노홍철       38
                               ..
힘들게     Adjective    유재석        1
힘들어져서   Adjective    박명수        1
힘들었다고   Adjective    노홍철        1
힘들었습니다  Adjective    박명수        1
힙합      Noun         정준하        1
Name: 카운트, Length: 6733, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소    품사          분류         
!      Punctuation 길        72
                   길씨 어머님    3
                   김제동      12
                   김태호       2
                   노홍철      38
...                        ...
힘들게    Adjective   유재석       1
힘들어져서  Adjective   박명수       1
힘들었다고  Adjective   노홍철       1
힘들었습니다 Adjective   박명수       1
힙합     Noun        정준하       1

[6733 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')